# 🧑‍💻 Amazon Nova built-in System Tools: Code Interpreter & Web Grounding

Amazon Nova 2 comes with two built-in system tools:
1. Code Interpreter allows Amazon Nova to execute code
2. Web Grounding enables Amazon Nova to search for real-time information on the web

This notebook guides you how to use the Code Interpreter & Web Grounding with Amazon Nova 2 Omni.

### 🎯 What You'll Build

By the end of this workshop, you will:
- ✅ Understand how the API for Code Interpreter & Web Grounding works
- ✅ Know how to use Code Interpreter & Web Grounding system tools
- ✅ Best practices for using system tools

### Prerequisites
- AWS Account
- [IAM Permissions](https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html) or an Amazon Bedrock [API Key](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_bedrock.html) to invoke Amazon Nova 2
- Python 3.8+

### Workshop Outline
1. Setup and Configuration
2. Quick start with Nova Code Interpreter
3. Sample Use Cases that require Code Execution
4. Quick start with Nova Web Grounding
5. Sample Use Cases that benefit from Web Grounding
6. Multi-Turn Conversation with Code Interpreter and Web Grounding 

## 1. Setup and Configuration

This section covers the initial setup required to use Amazon Nova's Code Interpreter.
We'll configure the AWS SDK (boto3) with appropriate settings for code execution tasks.

Before running this script, ensure you have the required dependencies installed:

In [ ]:
%pip install --quiet -r requirements.txt --quiet

In [ ]:
from IPython.display import Markdown, JSON

import boto3
from botocore.config import Config
import json 

The `create_boto3_client` helper function initializes a boto3 client for Amazon Bedrock Runtime
with custom timeout configurations. These extended timeouts are crucial for Code Interpreter
tasks that may take longer to execute.

In [ ]:
REGION = "us-west-2"
MODEL_ID = "us.amazon.nova-2-omni-v1:0"

In [ ]:
def create_boto3_client():
    client = boto3.client(
        "bedrock-runtime",
        region_name=REGION,
        config=Config(
            connect_timeout=3600,  # 60 minutes code interpreter might be more long running
            read_timeout=3600,     # 60 minutes
            retries={'max_attempts': 1}
        )
    )
    return client

In [ ]:
bedrock_client = create_boto3_client()

## 2. Quick start with Nova Code Interpreter

The Code Interpreter allows Amazon Nova to generate code that get's executed in a secure sandbox. 
This is useful for problems that require math, code writing or code review, and data analysis.

### Architecture of Code Interpreter

![](./img/Code-Interpreter.png)

### Example Use Cases
- Data Analysis, Data Visualization, Data-Driven Decision-Making
- Code Assistant, Code Reviewer
- Math
- Other use cases that require reproducible computation rather than probablistic next token prediction

### When not to use Code Interpreter
- If you need to retrieve real-time information from the web. For such use cases we recommend to use the Amazon Nova web search system tool.
- Currently, Amazon Nova Code Interpreter does not have access to your AWS resources. For automation tasks or to download from and upload data to an Amazon S3 bucket or database we recommend to use Amazon Bedrock AgentCore.


To use Web Grounding, specify the `nova_code_interpreter` systemTool in your `toolConfig` block:

```json
"toolConfig": {
    "tools": [
        {"systemTool": {"name": "nova_code_interpreter"}}
    ]
}
```

### Converse API

This section demonstrates how to use Amazon Nova's Code Interpreter or Web Grounding through the Converse API.
The Converse API provides a unified interface for interacting with foundation models,
including support for system tools like the Code Interpreter and Web Grounding.

The `construct_converse_payload` helper function constructs the payload required for the Converse API.
It includes the model ID, messages, and other parameters like temperature, max tokens, and tool configurations.

In [ ]:
def construct_converse_payload(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None):
    inferenceConfig = {
    }

    if max_tokens != None:
        inferenceConfig["maxTokens"] = max_tokens

    if temperature:
        inferenceConfig["temperature"] = temperature

    if top_p:
        inferenceConfig["topP"] = top_p

    tool_config = None
    if tools:
        tool_config = {
            "tools": tools
          }

    additional_request_fields = {}
    if reasoning_effort:
        reasoning_config = {
          "type": "enabled",
          "maxReasoningEffort": reasoning_effort
        }
        additional_request_fields["reasoningConfig"] = reasoning_config

    request = {
        "modelId":model, 
        "messages":messages, 
        "inferenceConfig":inferenceConfig,
        "toolConfig":tool_config,
        "additionalModelRequestFields":additional_request_fields
    }

    if system:
        request["system"] = system
    
    return request

The `amazon_bedrock_nova_converse` helper function is a wrapper that:
1. Constructs the request payload
2. Calls the Bedrock Runtime converse() API
3. Extracts the text response from the model
4. Returns both the text output and full response object

In [ ]:
def amazon_bedrock_nova_converse(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_converse_payload(model, messages, temperature, max_tokens, top_p, system, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse(**request)
        
        return model_response
        
    except Exception as e:
        print(type(e), e)
        return str(e), e

    

In [ ]:
def format_content_to_markdown(content_list, include_reasoning=False, include_tool_info=False):
    """
    Formats complex Nova response content to markdown with inline citations.
    
    Args:
        content_list: List of content items from the model response
        include_reasoning: Whether to include reasoning content in output
        include_tool_info: Whether to include tool use/result information
    
    Returns:
        Formatted markdown string with citations
    """
    markdown_parts = []
    sources = []
    citation_num = 0
    
    for item in content_list:
        if 'text' in item:
            markdown_parts.append(item['text'])
            
        elif 'citationsContent' in item:
            for citation in item['citationsContent'].get('citations', []):
                location = citation.get('location', {})
                if 'web' in location:
                    citation_num += 1
                    url = location['web'].get('url', '')
                    domain = location['web'].get('domain', url)
                    sources.append({'num': citation_num, 'url': url, 'domain': domain})
                    markdown_parts.append(f'[{citation_num}]')
                    
        elif 'reasoningContent' in item:
            if include_reasoning:
                reasoning = item['reasoningContent'].get('reasoningText', {}).get('text', '')
                if reasoning:
                    markdown_parts.append(
                        f'\n\n<span><span style="color: rgb(150, 34, 73); font-weight: bold;">&lt;details&gt;</span><span style="color: black; font-weight: normal;">\nReasoning\n\n{reasoning}\n\n</span><span style="color: rgb(150, 34, 73); font-weight: bold;">&lt;/details&gt;</span><br><br></span>\n\n'
                    )
                    
        elif 'toolUse' in item:
            if include_tool_info:
                tool = item['toolUse']
                name = tool.get('name', 'unknown')
                query = tool.get('input', {}).get('query', '')
                markdown_parts.append(f'\n\n&gt; **Tool:** `{name}`\n&gt; **Query:** "{query}"\n\n')
                
        elif 'toolResult' in item:
            if include_tool_info:
                status = item['toolResult'].get('status', '')
                if status:
                    markdown_parts.append(f'&gt; *Status: {status}*\n\n')
    
    # Combine text parts
    output = ''.join(markdown_parts)
    
    # Add sources section with deduplication
    if sources:
        # Group citation numbers by URL
        url_to_nums = {}
        url_to_domain = {}
        for src in sources:
            url = src['url']
            if url not in url_to_nums:
                url_to_nums[url] = []
                url_to_domain[url] = src['domain']
            url_to_nums[url].append(str(src['num']))
        
        output += '\n\n---\n\n**Sources:**\n'
        for url, nums in url_to_nums.items():
            nums_str = ', '.join(nums)
            domain = url_to_domain[url]
            # Use bold formatting instead of escaped brackets
            output += f'\n- **[{nums_str}]** [{domain}]({url})'
    
    return output

### Invoke Amazon Nova with Code Interpreter with the Converse API

This example demonstrates the basic workflow of using Code Interpreter with the converse API. To give Amazon Nova access to the Code Interpreter you specify the `systemTool` parameter in the tools config. 

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Print hello world using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]
output = format_content_to_markdown(content)

#### Understanding the Response

The model response contains multiple layers of information. Let's examine each part.
Here is the final output from Amazon Nova:

In [ ]:
print(output)

In addition to the final output, you can examine the model response to understand
what code was executed in the Code Interpreter. This is valuable for:
- Debugging: See exactly what code was generated
- Learning: Understand how the model approaches problems
- Auditing: Track what code is being executed in your application

Extract tool use and tool result from the response content:

In [ ]:
tool_use = next((trace for trace in content if "toolUse" in trace), None)
tool_result = next((trace for trace in content if "toolResult" in trace), None)

The `toolUse` block shows what code Amazon Nova generated and sent to the Code Interpreter. The `toolUse` > `input` > `code` property contains the code that got executed in the Code Interpreter.

In [ ]:
JSON(tool_use, expanded=True)

The output of the Code Interpreter tool contains the logs from `stdOut` in the Code Interpreter environment and also any errors that occured in the Code Interpreter environment.

In [ ]:
JSON(tool_result, expanded=True)

### Converse API with Streaming

Code Interpreter also works with streaming the response. Streaming allows you to receive the model's response incrementally as it's generated, rather than waiting for the complete response.

The `process_stream` helper function handles receiving the stream of output blocks.

In [ ]:
def process_stream(stream_response):
    is_reasoning = False
    is_text = False
    is_toolUse = False
    is_tool_result = False
    
    model_output = []  # List of content items matching format_content_to_markdown input
    
    # Temporary accumulators for streaming content
    current_text = ""
    current_reasoning = ""
    current_tool_use = None
    current_tool_use_input = ""
    
    def finalize_text():
        nonlocal current_text
        if current_text:
            model_output.append({'text': current_text})
            current_text = ""
    
    def finalize_reasoning():
        nonlocal current_reasoning
        if current_reasoning:
            model_output.append({
                'reasoningContent': {
                    'reasoningText': {'text': current_reasoning}
                }
            })
            current_reasoning = ""
    
    def finalize_tool_use():
        nonlocal current_tool_use, current_tool_use_input
        if current_tool_use:
            if current_tool_use_input:
                try:
                    current_tool_use['input'] = json.loads(current_tool_use_input)
                except json.JSONDecodeError:
                    current_tool_use['input'] = {'raw': current_tool_use_input}
            model_output.append({'toolUse': current_tool_use})
            current_tool_use = None
            current_tool_use_input = ""
    
    for message in stream_response["stream"]:
        if "contentBlockStart" in message:
            start = message["contentBlockStart"].get("start", {})
            
            # Finalize any previous content blocks
            finalize_text()
            finalize_reasoning()
            finalize_tool_use()
            
            if "toolUse" in start:
                is_toolUse = True
                is_text = False
                is_reasoning = False
                print("\n\n----MODEL RESPONSE: TOOL USE-----\n")
                current_tool_use = start["toolUse"].copy()
                current_tool_use_input = ""
                print(current_tool_use.get("name", ""))

        elif "contentBlockDelta" in message:
            delta = message["contentBlockDelta"]["delta"]

            # Check for regular text
            if "text" in delta:
                if not is_text:
                    finalize_reasoning()
                    print("\n\n----MODEL RESPONSE: TEXT-----\n")
                    is_text = True
                    is_reasoning = False
                    is_toolUse = False
                text_chunk = delta["text"]
                current_text += text_chunk
                print(text_chunk, end="", flush=True)

            # Check for reasoning content
            elif "reasoningContent" in delta and "text" in delta["reasoningContent"]:
                if not is_reasoning:
                    finalize_text()
                    print("\n\n----MODEL RESPONSE: REASONING-----\n")
                    is_text = False
                    is_reasoning = True
                    is_toolUse = False
                reasoning_chunk = delta["reasoningContent"]["text"]
                current_reasoning += reasoning_chunk
                print(reasoning_chunk, end="", flush=True)

            # Check for tool use input
            elif "toolUse" in delta:
                if "input" in delta["toolUse"]:
                    current_tool_use_input += delta["toolUse"]["input"]
                print(delta)
                
            # Check for tool result
            elif "toolResult" in delta:
                if not is_tool_result:
                    print("\n\n----MODEL RESPONSE: TOOL RESULT-----\n")
                    is_tool_result = True
                    is_toolUse = False
                model_output.append({'toolResult': delta["toolResult"]})
                print(delta)
            
            # Check for citations content
            elif "citationsContent" in delta:
                # Finalize text before citation so they stay paired
                finalize_text()
                is_text = False
                model_output.append({'citationsContent': delta["citationsContent"]})
                
        elif "contentBlockStop" in message:
            # Finalize current content block
            finalize_text()
            finalize_reasoning()
            finalize_tool_use()
            # Reset state flags
            is_text = False
            is_reasoning = False
            is_toolUse = False
            is_tool_result = False
            
        elif "messageStop" in message:
            print("\n\n----MODEL RESPONSE: messageStop-----\n")
            print(json.dumps(message, indent=4))
            
        elif "metadata" in message:
            print("\n\n----MODEL RESPONSE: metadata-----\n")
            print(json.dumps(message, indent=4))
    
    # Finalize any remaining accumulated content
    finalize_text()
    finalize_reasoning()
    finalize_tool_use()
    
    return model_output

Similar to the non-streaming version, but calls `converse_stream` instead. The function returns a stream object that must be iterated to receive events.

In [ ]:
def amazon_bedrock_nova_converse_streaming(model, messages, temperature, max_tokens, top_p = 0.9, system = None, reasoning_effort = None, tools = None, bedrock_rt_client = None ):  
    if not bedrock_rt_client:
        bedrock_rt_client = create_boto3_client()

    try:
        request = construct_converse_payload(model, messages, temperature, max_tokens, top_p, system, reasoning_effort, tools)

        model_response = bedrock_rt_client.converse_stream(**request)
        
        return model_response
        
    except Exception as e:
        print(type(e), e)
        return e

    

### Invoke Amazon Nova with Code Interpreter using the Converse Streaming API 

Next you will invoke the Converse Streaming API with Code Interpreter system tool configured.

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Calculate 7^6 using the code interpreter"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

response = amazon_bedrock_nova_converse_streaming(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
model_output = process_stream(response)

The final response from Amazon Nova is:

In [ ]:
output = format_content_to_markdown(model_output)

In [ ]:
print(output)

## 3. Sample Use Cases that require Code execution

This section demonstrates scenarios where Code Interpreter excels. These examples showcase the power of combining LLM reasoning with code execution for tasks that require precise computation.

#### Solving Polynomial Equations

Polynomial equations often have complex or multiple roots that are difficult to find even for reasoning models. Code Interpreter can use numerical methods (like NumPy's roots function) to find all solutions, including complex roots.

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Use Python to solve the polynomial equation: z^6 + z^4 + z^3 + z^2 + 1 = 0. Find the exact roots. Format your response in markdown."
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 5000
reasoning_effort = None

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]

output = format_content_to_markdown(content, include_reasoning=False, include_tool_info=False)

Here is the solution for the polynomial:

In [ ]:
Markdown(output)

#### Statistical Hypothesis Testing

Statistical analysis is a perfect use case for Code Interpreter because it requires:
- Precise mathematical calculations
- Standard statistical libraries (scipy, numpy)
- Amazon Nova's capability to interpret the results in context

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": """I have two datasets of student test scores. Perform a t-test to determine if there's a statistically significant difference between them at α=0.05 confidence level.

Group A: [78, 85, 92, 88, 79, 91, 84, 87, 90, 82]
Group B: [72, 75, 81, 69, 74, 78, 71, 76, 73, 70]
"""
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 10000
reasoning_effort = None

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]

output = format_content_to_markdown(content, include_reasoning=False, include_tool_info=False)

Here is the output of the statistical analysis that Amazon Nova conducted using the code interpreter:

In [ ]:
print("\nStatistical analysis results:")
Markdown(output)

---------

## 4. Quick start with Nova Web Grounding

Web Grounding enhances Amazon Nova models by connecting them to real-time information beyond their knowledge cutoff, resulting in more accurate and reliable responses with cited sources. This built-in tool automatically retrieves and incorporates publicly available information to reduce AI hallucinations and improve accuracy.

### How It Works

When Web Grounding is enabled, the model intelligently determines if a search is needed to improve the response. If required, it automatically performs one or more searches, analyzes the results, and synthesizes the information into a final response complete with citations to its sources. All searches stay within AWS infrastructure and never reach the broader internet, prioritizing trustworthy sources and filtering malicious content.

### Example Use Cases

- **Knowledge-Based Chat Assistants**: Provide up-to-date answers with source citations for customer inquiries
- **Research Assistants**: Access current information and factual data with transparent source attribution
- **Content Generation Tools**: Create content grounded in real-time, cited information
- **Customer Support Applications**: Deliver accurate responses based on the latest available information
- **News and Current Events**: Answer questions about recent developments with proper citations

### When Not to Use Web Grounding

- **For Private or Internal Data**: Web Grounding only accesses publicly available information. For proprietary data, use Retrieval Augmented Generation (RAG) with Amazon Bedrock Knowledge Bases.

### Availability and Requirements

Web Grounding is currently available in US regions.

To use Web Grounding, specify the `nova_grounding` systemTool in your `toolConfig` block:

```json
"toolConfig": {
    "tools": [
        {"systemTool": {"name": "nova_grounding"}}
    ]
}
```

**Important**: You must retain and display citations and links in the output you provide to your end users.

### Invoke Amazon Nova with Web Grounding with the Converse API

This example demonstrates the basic workflow of using Web Grounding with the converse API. To give Amazon Nova access to Web Grounding you specify `nova_grounding` in the `systemTool` parameter in the tools config. 

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "What is the price of Amazon Stock?"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]
output = format_content_to_markdown(content)

#### Understanding the Response

Let's take a look at the different items that are returned in the response.

Here is the **final response** that Amazon Nova generated:

In [ ]:
print(output)

You can also **see more details** about the web grounding tool call.
The response from the output contains the query that Amazon Nova constructed to send to the web grounding system in the `toolUse` > `input` > `query` parameter.

If you take a look at the response from the API before formatting it to Markdown you can see how the text that Amazon Nova has generated is interleaved with the `citationsContent` blocks.

In [ ]:
content

### Invoke Amazon Nova with Web Grounding using the Converse Streaming API 

Next you will invoke the Converse Streaming API with Web Grounding system tool configured.

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "What is the weather in Seattle today?"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = 1000
reasoning_effort = None

response = amazon_bedrock_nova_converse_streaming(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)
model_output = process_stream(response)

In [ ]:
output = format_content_to_markdown(model_output)

The final response is:

In [ ]:
print(output)

## 5. Sample Use Cases that benefit from Web Grounding

This section demonstrates scenarios where Web Grounding is useful. Before Web Grounding such capabilities would require maintaining a custom Retrieval-Augmented Generation (RAG) pipeline of financial reports or the integration with another API. Now with Web Grounding as a system tool public real-time information can be retrieved out of the box.

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Find the latest earnings report from Amazon and analyze main trends. Respond in markdown"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = None
reasoning_effort = "medium"

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

In [ ]:
print(format_content_to_markdown(model_response["output"]["message"]["content"], include_reasoning=False, include_tool_info=False))

You can combine the reasoning capabilities of Amazon Nova Omni with Web Grounding to have Amazon Nova reason through conflicting or divisive information:

In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "Pineapple on Pizza."
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

temperature = None
top_p = None
max_tokens = None
reasoning_effort = "high"

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, reasoning_effort=reasoning_effort, tools=tools, bedrock_rt_client=bedrock_client)

Below is the final response:

In [ ]:
print(format_content_to_markdown(model_response["output"]["message"]["content"], include_reasoning=False, include_tool_info=False))

## 6. Multi-Turn Conversation with Code Interpreter and Web Grounding 

This section demonstrates a realistic business scenario where a coffee shop owner uses data analysis to reduce waste and make informed decisions.

In this example, we'll simulate a conversation with María, a coffee shop owner who wants to understand and reduce her daily pastry waste. The conversation progresses from initial problem identification to data analysis and decision-making.

The multi-turn conversation combines both Code Interpreter with Web Grounding  


In [ ]:
messages = [
    {
      "role": "user",
      "content": [
        {
          "text": "I'm throwing away so many pastries at the end of each day. It breaks my heart and wastes money. What can I do about this?"
        }
      ]
    }
  ]

tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        },
    {
            "systemTool": {
                "name": "nova_grounding"
            }
        }
    ]

system = [{ "text": "You are a helpful assistant. Use the Code Interpreter or Web Grounding tool when available and appropriate. Keep your answers short. Format your responses in Markdown." }]

temperature = 0.7
top_p = 0.9
max_tokens = 1000

model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, system=system, tools=tools, bedrock_rt_client=bedrock_client)

content = model_response["output"]["message"]["content"]
output = format_content_to_markdown(content)

Turn 1 - Initial consultation:

In [ ]:
print(output)

In [ ]:
messages.append({
    "role": "assistant",
    "content": [{"text": output}]
})

messages.append({
    "role": "user",
    "content": [{
        "text": "That's helpful! I'm wondering if my waste is actually abnormal though. I throw away about 8-12 pastries per day. I order the same amount every week from my supplier—4 dozen croissants, 3 dozen muffins, 2 dozen danishes. What's the typical waste percentage for bakeries, and how do I compare?"
    }]
})

In [ ]:
model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]
output = format_content_to_markdown(content)

Turn 2 - Understanding the pattern:

In [ ]:
print(output)

In [ ]:
tools = [
        {
            "systemTool": {
                "name": "nova_code_interpreter"
            }
        }
    ]

In [ ]:
messages.append({
    "role": "assistant",
    "content": [{"text": output}]
})

messages.append({
    "role": "user",
    "content": [{
        "text": """Okay, I tracked what I throw away for a week. Here's what I threw away each day:

Monday: 6 croissants, 4 muffins

Tuesday: 5 croissants, 3 muffins

Wednesday: 8 croissants, 5 muffins, 2 danishes

Thursday: 4 croissants, 2 muffins, 1 danish

Friday: 2 croissants, 1 muffin

Saturday: 1 croissant, 0 muffins

Sunday: 3 croissants, 2 muffins

Each croissant costs me $1.50, muffins $1.20, and danishes $1.80. Calculate my total weekly losses, break it down by day and pastry type, and show me which days are worst. Also, if I reduced my orders to match this waste pattern, how much would I save annually?"""
    }]
})

In [ ]:
model_response = amazon_bedrock_nova_converse(MODEL_ID, messages, temperature, max_tokens, top_p, tools=tools, bedrock_rt_client=bedrock_client)
content = model_response["output"]["message"]["content"]
output = format_content_to_markdown(content)

Turn 3 - Waste analysis with calculations:

In [ ]:
print(output)

Let's examine the code that was executed:

In [ ]:
content = model_response['output']['message']['content']
tool_use = next((trace for trace in content if "toolUse" in trace), None)
if tool_use and "input" in tool_use["toolUse"]:
    print("Code executed by Amazon Nova:")
    print(tool_use["toolUse"]["input"]["snippet"])

## System Tools Pricing

You can review the pricing for Web Grounding and Code Interpreter in the Amazon Nova model pricing details sections on [https://aws.amazon.com/bedrock/pricing/](https://aws.amazon.com/bedrock/pricing/).

## 🎯 Workshop Summary

You've learnt how to use built in system tools like Code Interpreter and Web Grounding with Amazon Nova 2 Omni.

### Additional Resources

- [User Guide for Amazon Nova - Web Grounding](https://docs.aws.amazon.com/nova/latest/userguide/grounding.html)
- [User Guide for Amazon Nova - Code Interpreter](https://docs.aws.amazon.com/nova/latest/userguide/grounding.html)

Great job completing this workshop! 🎉